### Learning to make a tensorflow dataset

In [20]:
# creating tfrecord file

import tensorflow as tf
import os
from matplotlib.image import imread

# Define the paths to your image folders
folder_paths = {
    'Bhaktapur-Durbar-Square': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Bhaktapur-Durbar-Square',
    'Bouddanath': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Bouddanath',
    'Pashupatinath': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Pashupatinath',
    'Patan-Durbar-Square': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Patan-Durbar-Square',
    'Swyambunath': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Swyambunath'
}

# Define the output TFRecord file name
output_directory = '/Users/nirajanpaudel17/Documents/Python/Major-Project'
output_filename = os.path.join(output_directory, 'labeled_temples_images.tfrecord')

# Initialize an empty list to store the labeled images
labeled_images = []

# Iterate over each class folder
for class_name, folder_path in folder_paths.items():
    
    # Get the list of image filenames in the current class folder
    image_filenames = os.listdir(folder_path)
    

    # Iterate over each image in the class folder
    for image_filename in image_filenames:
        # Create the full path to the image file
        image_path = os.path.join(folder_path, image_filename)

        try:
            # Read the image file
            image = tf.io.read_file(image_path)

            # Decode the image file
            image = tf.io.decode_jpeg(image)
            
            image = tf.io.encode_jpeg(image).numpy()


            # Create a labeled example
            example = tf.train.Example(features=tf.train.Features(feature={
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                'class': tf.train.Feature(bytes_list=tf.train.BytesList(value=[class_name.encode()]))
            }))

            # Append the labeled example to the list
            labeled_images.append(example)

        except tf.errors.InvalidArgumentError:
            print('Skipping unsupported image:', image_path)

# Create a writer for the TFRecord file
with tf.io.TFRecordWriter(output_filename) as writer:
    # Write each labeled example to the TFRecord file
    for example in labeled_images:
        writer.write(example.SerializeToString())

# # Print the number of labeled images
print('Number of labeled images:', len(labeled_images))


Number of labeled images: 1639


In [98]:
import tensorflow as tf


def parse_tfrecord_fn(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'class': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    image = tf.image.decode_jpeg(example['image'], channels=3)  # Modify the decoding function as per image format
    image = tf.cast(image, tf.float32) / 255.0
    
    label = tf.cast(example['class'], tf.string)
    
    return image, label


# Specify the path to the TFRecord file(s)
tfrecord_files = ['/Users/nirajanpaudel17/Documents/Python/Major-Project/labeled_temples_images.tfrecord']

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_files)

# dataset_size = tf.data.experimental.cardinality(dataset).numpy()
# print("Dataset size:", dataset_size)

# Apply parsing function to the dataset
dataset = dataset.map(parse_tfrecord_fn)

# Print the first few examples in the dataset
for image, label in dataset.take(1):
    # Process or use the image and label as needed
    print(image.shape, label)


(300, 300, 3) tf.Tensor(b'Bhaktapur-Durbar-Square', shape=(), dtype=string)


In [89]:
# splitting the datset into train and test set

dataset = dataset.shuffle(buffer_size=1000, seed=42)

# Split the dataset into train and test sets

num_samples = int(dataset.cardinality().numpy())

# Split the dataset into train, validation, and test sets
train_size = int(0.8 * num_samples)  # 80% for training
val_size = int(0.1 * num_samples)  # 10% for validation
test_size = num_samples - train_size - val_size  # Remaining 10% for testing

# Create train, validation, and test datasets
train_dataset = dataset.take(train_size)
remaining_dataset = dataset.skip(train_size)
val_dataset = remaining_dataset.take(val_size)
test_dataset = remaining_dataset.skip(val_size)

# Print the number of samples in each set
print("Train set size:", train_size)
print("Validation set size:", val_size)
print("Test set size:", test_size)

Train set size: -1
Validation set size: 0
Test set size: -1


In [77]:
images_to_train = []
labels_of_train = []

for image, label in train_dataset:
    
    images_to_train.append(image)
    labels_of_train.append(label)
    
images_to_test = []
labels_of_test = []

for image, label in test_dataset:
    
    images_to_test.append(image)
    labels_of_test.append(label)

In [51]:
images_to_train[0].shape

TensorShape([300, 300, 3])

In [55]:
labels_of_train[300]

<tf.Tensor: shape=(), dtype=string, numpy=b'Pashupatinath'>

In [ ]:
# from tensorflow import keras

# model = keras.applications.resnet50.ResNet50(weights="imagenet")

# images_resized = tf.image.resize(images_to_train, [224, 224])

# # resnest requires image size of [224,224]

# # images_resized[0].shape

# # resnet provides a preprocess function to make the images compatible with it. Also, pixel range from 0 to 255

# inputs = keras.applications.resnet50.preprocess_input(images_resized * 255) 

# Y_proba = model.predict(inputs)

# import numpy as np
# np.argmax(Y_proba[100])

# import matplotlib.pyplot as plt
# # plt.imshow(images_to_train[100])

In [90]:
# Now let's use the Xception model 

def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image) 
    return final_image, label

In [91]:
batch_size = 32
train_set = train_dataset.shuffle(1000).repeat()
train_set = train_dataset.map(preprocess).batch(batch_size).prefetch(1)
valid_set = val_dataset.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_dataset.map(preprocess).batch(batch_size).prefetch(1)

In [92]:
from tensorflow import keras

n_classes = 5
base_model = keras.applications.xception.Xception(include_top=False,weights='imagenet')
# this excludes the global average pooling layer and the dense output layer.
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes,activation='softmax')(avg)
model = keras.models.Model(inputs=base_model.input,outputs=output)


for layer in base_model.layers: 
    layer.trainable = False
    

optimizer = keras.optimizers.legacy.SGD(learning_rate=0.2, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])


In [99]:
 history = model.fit(train_set,
                        steps_per_epoch=int(0.75 * 1500 / batch_size),
                        validation_data=valid_set,
                        validation_steps=int(0.15 * 1500 / batch_size),
                        epochs=5)

Epoch 1/5


2023-07-11 13:24:35.596862: W tensorflow/core/framework/op_kernel.cc:1805] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "/Users/nirajanpaudel17/anaconda3/envs/tnfenv/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/nirajanpaudel17/anaconda3/envs/tnfenv/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/nirajanpaudel17/anaconda3/envs/tnfenv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/nirajanpaudel17/anaconda3/envs/tnfenv/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/nirajanpaudel17/anaconda3/envs/tnfenv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/nirajanpaudel17/anaconda3/envs/tnfenv/lib/python3.10/site-packages/tornado/platform/asyncio.py", lin